In [47]:
# Somehow in mac, it cant find local packages
# Run this to solve no local modules found in mac
import os
import sys

# Get the project root directory (assuming your notebook is in a subdirectory)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [48]:
# Somehow in mac, it cant find local packages
# Run this to solve no local modules found in mac
import os
import sys

# Get the project root directory (assuming your notebook is in a subdirectory)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


# Python lib
import asyncio
import os

# Autogen-0.4
from autogen_agentchat.agents import AssistantAgent

from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Local
from utils import get_dataset_profile, get_columns_sample, initialize_individual_chat, jsonify_prompt
from prompts import data_dict_summarizer_prompt

In [49]:
root = '../sheets/mysql/'
files: list[str] = os.listdir(path=root)

# Uses utils/get_dataset_profile
# tasks = [get_dataset_profile(root, file_name = file, output_format='json') for file in files]

# # Uses utils/get_columns_sample
tasks = [get_columns_sample(root, file_name = file) for file in files]

results = await asyncio.gather(*tasks)
result_dict = zip(files, results)
results = list(result_dict)

In [57]:
data_dict_generator = "qwen2.5:32b-instruct-q8_0"
data_dict_summarizer = "deepseek-r1:32b-qwen-distill-q8_0-131072"

# Common config
llm_base_url = "http://34.204.63.234:11434/v1"
api_key = "none"
model_info_generator =  {
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "family": "Qwen2.5"
    }
model_info_summarizer =  {
        "vision": False,
        "function_calling": False,
        "json_output": False,
        "family": "DeepSeek-r1"
    }

# Create reasoning config
# Just generate the data dict based on obervered facts
data_dict_summarizer_client = OpenAIChatCompletionClient(
    frequency_penalty=0.2, 
    logit_bias=None, 
    max_tokens=128000,
    presence_penalty=0.5, 
    seed=42, 
    temperature=0.8, 
    top_p=0.95, 
    model=data_dict_summarizer,
    api_key='none',  
    model_info=model_info_summarizer, 
    base_url='http://34.204.63.234:11434/v1'
    )

# Just generate the data dict based on obervered facts
data_dict_generator_client = OpenAIChatCompletionClient(
    frequency_penalty=0.4, 
    logit_bias=None, 
    max_tokens=2048, 
    presence_penalty=0.3, 
    response_format={"type": "json_object"},
    seed=42, 
    temperature=0.2, 
    top_p=0.7, 
    model=data_dict_generator,
    api_key='none', 
    model_info=model_info_generator, 
    base_url='http://34.204.63.234:11434/v1')



In [51]:
# Create a list of tasks for all agents
tasks = [initialize_individual_chat(filename=results[index][0], metadata=results[index][1], data_dict_generator_client=data_dict_generator_client) for index in range(len(results))]

# Execute all agent chat tasks concurrently
responses = await asyncio.gather(*tasks)
print(f"\nResponses received: {len(responses)}")
print(responses)



Responses received: 8
[Response(chat_message=TextMessage(source='File_handler_products_csv', models_usage=RequestUsage(prompt_tokens=828, completion_tokens=796), content='{\n  "Data Dictionary": {\n    "filename": "products.csv",\n    "columns": [\n      {\n        "Column": "productCode",\n        "Description": "Unique identifier for each product.",\n        "Format": "Identifier: Starts with a letter followed by numbers (e.g., S10_1678).",\n        "Nullable": false,\n        "Sample Values": ["S10_1678", "S10_1949", "S10_2016"]\n      },\n      {\n        "Column": "productName",\n        "Description": "Name of the product.",\n        "Format": "Text: Contains free-form text strings representing names (e.g., 1969 Harley Davidson Ultimate Chopper).",\n        "Nullable": false,\n        "Sample Values": ["1969 Harley Davidson Ultimate Chopper", "1952 Alpine Renault 1300", "1996 Moto Guzzi 1100i"]\n      },\n      {\n        "Column": "productLine",\n        "Description": "Categor

In [52]:
# Jsonify and Save log
generated_data_dict, log_file_name = await jsonify_prompt(responses)


Data saved to: generation_log/Log_202502051638.log


In [58]:
# Initialize data dict ingestion agent
final_agent = AssistantAgent(name="Data_Dictionary_Analytics_Suggester",
                             model_client=data_dict_summarizer_client,
                             system_message=data_dict_summarizer_prompt()
)

response = await final_agent.on_messages(
        [TextMessage(content=f"{generated_data_dict}\n\nReview the above data dictionary. First by stating the list of data dictionaries received.\n\n",source="user")], None
    )
response

Response(chat_message=TextMessage(source='Data_Dictionary_Analytics_Suggester', models_usage=RequestUsage(prompt_tokens=5915, completion_tokens=1148), content="<think>\nAlright, so I've got this problem where I need to analyze some data dictionaries and come up with analytics use cases based on that analysis. The instructions are pretty detailed, so I want to make sure I understand everything properly before diving in.\n\nFirst off, there are eight different CSV files provided: products, orders, employee, office, customer, productlines, payments, and orderdetails. Each of these has its own set of columns with specific descriptions and formats. My goal is to look at how these datasets relate to each other and then suggest analytics use cases that can leverage those relationships.\n\nOkay, let's break it down step by step as per the instructions:\n\n1. **Review All Provided Data Dictionaries**: I need to go through each dataset carefully. That means looking at all the columns in products

In [22]:
print(response.chat_message.content)

<think>
Alright, so I need to analyze these data dictionaries and come up with some analytics use cases. Let me start by reviewing each dataset provided.

First, there's products.csv which includes details about each product like productCode, productName, productLine, and more financial metrics like buyPrice and MSRP. This seems useful for understanding product performance and profitability.

Next is orders.csv, containing order information such as orderNumber, dates (orderDate, requiredDate, shippedDate), status, comments, and customerNumber. This can help in analyzing the order fulfillment process and customer behavior related to orders.

Employee.csv has data about employees including their contact info, office details, job titles, and who they report to. This could be used for workforce analytics or understanding organizational structure.

Office.csv provides information about different offices like city, phone, address lines, state, country, postal code, and territory. Useful for 